In [1]:
COLAB_FLAG = False   # whether running on colab or locally on computer

In [2]:
if COLAB_FLAG:
    !pip install pydicom==2.1.2
    !pip install monai seaborn sentence_transformers
    !git clone 'https://github.com/mille055/AIPI540_individual_project.git'

    


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import torchvision
import pydicom
#import monai
import pickle
import glob
import sklearn
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, recall_score, plot_confusion_matrix, ConfusionMatrixDisplay
from sklearn.linear_model import LogisticRegression

import seaborn as sns
from datetime import datetime
from sentence_transformers import SentenceTransformer

from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn

import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import time
import os
import copy
from pprint import pprint
from fastai.basics import delegates
from fastcore.parallel import parallel
from fastcore.utils import gt
from fastcore.foundation import L

# import monai
# from monai.data import DataLoader, ImageDataset
# from monai.transforms import AddChannel, Compose, RandRotate90, Resize, ScaleIntensity, EnsureType
from pydicom.dataset import Dataset as DcmDataset
from pydicom.tag import BaseTag as DcmTag
from pydicom.multival import MultiValue as DcmMultiValue
import sys
import importlib
import warnings
warnings.filterwarnings("ignore")

In [4]:
#local imports
if COLAB_FLAG:
    sys.path.append('/content/AIPI540_individual_project/scripts/')
    train_datafile = '/content/AIPI540_individual_project/data/trainfiles.csv'
    val_datafile = '/content/AIPI540_individual_project/data/valfiles.csv'
    test_datafile = '/content/AIPI540_individual_project/data/testfiles.csv'

else: # running locally
    sys.path.append('/Users/cmm/Documents/GitHub/AIPI540_individual_project/scripts/')
    train_datafile = '../data/trainfiles.csv'
    val_datafile = '../data/valfiles.csv'
    test_datafile = '../data/testfiles.csv'

### local imports ###
from config import file_dict, feats, feats_to_keep, column_lists, RF_parameters, classes, model_paths
from config import abd_label_dict, val_list, train_val_split_percent, random_seed, data_transforms
from config import sentence_encoder, series_description_column
from utils import *
#from train_pixel_model import train_pix_model, test_pix_model, ImgDataset, image_to_tensor, get_pixel_preds_and_probs
#from train_meta_model import train_fit_parameter_trial, train_meta_model, calc_feature_importances, get_meta_probs, meta_inference
#from train_text_model import train_text_log_model, load_text_data, get_nlp_inference
from NLP.NLP_inference import get_NLP_inference
from NLP.NLP_training import train_NLP_model
from cnn.cnn_dataset import ImgDataset
from cnn.cnn_inference import image_to_tensor, pixel_inference, test_pix_model, load_pixel_model, visualize_results
from cnn.cnn_model import CustomResNet50
from cnn.cnn_data_loaders import get_data_loaders
from metadata.meta_inference import meta_inference, calc_feature_importances
from metadata.meta_training import train_fit_parameter_trial, train_meta_model, evaluate_meta_model
from fusion_model.fus_model import FusionModel
from fusion_model.fus_inference import get_fusion_inference
from model_container import ModelContainer
from process_tree import Processor, write_labels_into_dicom
# from AIPI540_individual_project.scripts.train_pixel_model import train_model
# from AIPI540_individual_project.scripts.train_text_model import load_text_data, train_text_model, list_incorrect_text_predictions
# from AIPI540_individual_project.scripts.utils import *

In [16]:
fusion_model = FusionModel

class PartialFusionModel(nn.Module):
    def __init__(self, model1, model2, num_classes):
        super(PartialFusionModel, self).__init__()
        self.model1 = model1
        self.model2 = model2
        #self.model3 = model3
        self.fusion_layer = nn.Linear(num_classes * 2, num_classes)

    def forward(self, x1, x2):
        
        x = torch.cat((x1, x2), dim=0)
        x = self.fusion_layer(x)

        return x
    
partial_fusion_model = PartialFusionModel



In [18]:
with open('../models/fusion_model041623.pkl', 'rb') as file:
    fusion = pickle.load(file)

print(fusion)

FusionModel(
  (model2): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
       

In [ ]:
### fix my fusion model issue:; have to change from a pickled model to saved weights
def save_fusion_model_weights(pickle_path, weight_path):
    with open(pickle_path, 'rb') as file:
        fusion_model = pickle.load(file)
    torch.save(fusion_model.state_dict(), weight_path)

save_fusion_model_weights(model_paths['fusion'], '../models/fusion_saved_weights042123.pth')
save_fusion_model_weights('../models/meta_and_pixel_fusion_model041623.pkl', '../models/fusion_saved_weights_no_nlp042123.pth')

In [ ]:
### in the fusion model creation code:

Xtrain1 = fusion_train_df.meta_probs.values

Xtrain2 = fusion_train_df.pixel_probs.values
Xtrain3 = fusion_train_df.nlp_probs.values

Xtrain1 = np.stack(Xtrain1, axis=0)
Xtrain2 = np.stack(Xtrain2, axis=0)
Xtrain3 = np.stack(Xtrain3, axis=0)

out_features = 19


y_train_fusion = np.array([classes.index(x) for x in fusion_train_df.true])


In [ ]:
# pickling the dataframes which have undergone inference from the base models

# fusion_train_df.to_pickle('../data/fusion_train.pkl')
# fusion_val_df.to_pickle('../data/fusion_val.pkl')
# fusion_test_df.to_pickle('../data/fusion_test.pkl')


In [ ]:
# class FusionModel(nn.Module):
#     def __init__(self, model1, model2, model3, num_classes):
#         super(FusionModel, self).__init__()
#         self.model1 = model1
#         self.model2 = model2
#         self.model3 = model3
#         self.fusion_layer = nn.Linear(num_classes * 3, num_classes)

#     def forward(self, x1, x2, x3):
        
#         x = torch.cat((x1, x2, x3), dim=0)
#         x = self.fusion_layer(x)

#         return x

In [ ]:
class PartialFusionModel(nn.Module):
    def __init__(self, model1, model2, num_classes):
        super(PartialFusionModel, self).__init__()
        self.model1 = model1
        self.model2 = model2
        #self.model3 = model3
        self.fusion_layer = nn.Linear(num_classes * 2, num_classes)

    def forward(self, x1, x2):
        
        x = torch.cat((x1, x2), dim=0)
        x = self.fusion_layer(x)

        return x
    

# ## since I merged this into FusionModel with conditions after saving the weights, i need to try to recover the weights into 
# ## the new mdoel
# # load components


# #pmodel = PartialFusionModel(model_pix, meta_model, num_classes=19)

# from config import model_paths
# model_container = ModelContainer()

# with open(model_paths['fusion_no_nlp'], 'rb') as file:
#     partial_fusion_model = pickle.load(file)

# fusion_model_with_partial_weights = FusionModel(models, num_classes=19, include_nlp=False)
# fusion_model_with_partial_weights.fusion_layer.weight = nn.Parameter(partial_fusion_model.fusion_layer.weight)

# # Save only the model weights (state_dict)
# torch.save(fusion_model_with_partial_weights.state_dict(), '../models/fusion_model_no_nlp042123.pth')



In [ ]:
# fusion_model = FusionModel


# num_classes = 19  # Number of classes
# pixel_model = model
# fusion_model = FusionModel(meta_model, pixel_model, NLP_model, num_classes)
# meta_and_pixel_fusion_model = PartialFusionModel(meta_model, pixel_model, num_classes)
# criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(fusion_model.parameters(), lr=0.001)
# p_optimizer = torch.optim.Adam(meta_and_pixel_fusion_model.parameters(), lr=0.001)

In [ ]:
# num_epochs = 20
# batch_size = 32

# for epoch in range(num_epochs):
#     for i in range(0, len(y_train_fusion), batch_size):
#         actual_batch_size = min(batch_size, len(Xtrain1) - i)
#         X1_batch = torch.tensor(np.array(Xtrain1[i:i+actual_batch_size], dtype=np.float32), dtype=torch.float32)
#         X2_batch = torch.tensor(np.array(Xtrain2[i:i+actual_batch_size], dtype=np.float32), dtype=torch.float32)
#         X3_batch = torch.tensor(np.array(Xtrain3[i:i+actual_batch_size], dtype=np.float32), dtype=torch.float32)
#         y_batch = torch.tensor(y_train_fusion[i:i+actual_batch_size], dtype=torch.long)

#         # print(f"X1_batch shape: {X1_batch.shape}")
#         # print(f"X2_batch shape: {X2_batch.shape}")
#         # print(f"X3_batch shape: {X3_batch.shape}")
#         # print(f"y_batch shape: {y_batch.shape}")


#         optimizer.zero_grad()
#         outputs = fusion_model(X1_batch, X2_batch, X3_batch)
#         loss = criterion(outputs, y_batch)
#         loss.backward()
#         optimizer.step()

#     print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item()}')


In [ ]:
# Xtest1 = fusion_test_df.meta_probs.values
# Xtest2 = fusion_test_df.pixel_probs.values
# Xtest3 = fusion_test_df.nlp_probs.values

# Xtest1 = np.stack(Xtest1, axis=0)
# Xtest2 = np.stack(Xtest2, axis=0)
# Xtest3 = np.stack(Xtest3, axis=0)

# out_features = 19

# actual_classes = [0,2,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,23,25]
# y_test_fusion = np.array([actual_classes.index(x) for x in fusion_test_df.true])

In [ ]:
# import torch.nn.functional as F
# # Evaluate the model with the test data
# with torch.no_grad():
#     X1_test_t = torch.tensor(Xtest1, dtype=torch.float32)
#     X2_test_t = torch.tensor(Xtest2, dtype=torch.float32)
#     X3_test_t = torch.tensor(Xtest3, dtype=torch.float32)
#     y_test_t = torch.tensor(y_test_fusion, dtype=torch.long)

#     outputs = fusion_model(X1_test_t, X2_test_t, X3_test_t)
#     f_probabilities = F.softmax(outputs, dim=1)  # Apply softmax to the outputs

#     _, f_predicted = torch.max(outputs, 1)
#     correct = (f_predicted == y_test_t).sum().item()
#     f_accuracy = correct / len(y_test_t) * 100

# print(f'Test accuracy: {f_accuracy}%')

In [ ]:
# def process_batch(df, data_dir, destination_folder, write_labels=True):

#     df1 = df.copy()
#     #print('In batch, columns are: ', df1.columns)
#     batch = df1.groupby('patientID').apply(lambda x: process_patient(x, data_dir, destination_folder, write_labels))
    
#     # print('writing labels into dicom in location ', dest_name)
#     # for filename in batch.fname:

#     return batch

# def process_patient(patient_df, data_dir, destination_folder, write_labels):
#     processed_exams = patient_df.groupby('exam').apply(lambda x: process_exam(x, data_dir, destination_folder, write_labels))
#     return processed_exams


# def process_exam(exam_df, data_dir, destination_folder, write_labels):
#     # Group exam data by series and apply the process_series function
#     processed_series = exam_df.groupby('series').apply(lambda x: process_series(x, data_dir, destination_folder, write_labels))

#     #result = pd.concat(processed_series)
#     result = processed_series
#     return result
    
# def process_series(series_df, data_dir, destination_folder, write_labels, selection_fraction=0.5):
#     # Sort the dataframe by file_info (or another relevant column)
#     sorted_series = series_df.sort_values(by='fname')

#     # Find the middle image index
#     middle_index = int(len(sorted_series) * selection_fraction)

#     # Get the middle image
#     middle_image = sorted_series.iloc[middle_index]

#     predicted_series_class, predicted_series_confidence = get_fusion_inference(middle_image)

#     sorted_series['predicted_class'] = predicted_series_class
#     sorted_series['prediction_confidence'] = np.round(predicted_series_confidence, 2)

#     #save_path = f'/volumes/cm7/processed/modified/{series_df.patientID}/{series_df.exam}/{series_df["series"]}/'

#      # Define the save path relative to data_dir
#     relative_path = os.path.relpath(series_df.fname.iloc[0], data_dir)
#     save_path = os.path.join(data_dir, destination_folder, os.path.dirname(relative_path))

#     if not os.path.exists(save_path):
#         os.makedirs(save_path)

#     if write_labels:
#         #print('writing new data into', save_path)
#         write_labels_into_dicom(sorted_series, label_num=predicted_series_class,
#                             conf_num=np.round(predicted_series_confidence, 3), path=save_path)

#     return sorted_series

In [ ]:
def process_batch(df, data_dir, destination_folder, write_labels=True):

    df1 = df.copy()
    #print('In batch, columns are: ', df1.columns)
    batch = df1.groupby('patientID').apply(lambda x: process_patient(x, data_dir, destination_folder, write_labels))
    
    # print('writing labels into dicom in location ', dest_name)
    # for filename in batch.fname:

    return batch

def process_patient(patient_df, data_dir, destination_folder, write_labels):
    processed_exams = patient_df.groupby('exam').apply(lambda x: process_exam(x, data_dir, destination_folder, write_labels))
    return processed_exams


def process_exam(exam_df, data_dir, destination_folder, write_labels):
    # Group exam data by series and apply the process_series function
    processed_series = exam_df.groupby('series').apply(lambda x: process_series(x, data_dir, destination_folder))

    #result = pd.concat(processed_series)
    result = processed_series
    return result
    
def process_series(series_df, data_dir, destination_folder, write_labels, selection_fraction=0.5):
    # Sort the dataframe by file_info (or another relevant column)
    sorted_series = series_df.sort_values(by='fname')

    # Find the middle image index
    middle_index = int(len(sorted_series) * selection_fraction)

    # Get the middle image
    middle_image = sorted_series.iloc[middle_index]

    predicted_series_class, predicted_series_confidence = get_fusion_model_prediction(middle_image)

    sorted_series['predicted_class'] = predicted_series_class
    sorted_series['prediction_confidence'] = np.round(predicted_series_confidence, 2)

    #save_path = f'/volumes/cm7/processed/modified/{series_df.patientID}/{series_df.exam}/{series_df["series"]}/'

     # Define the save path relative to data_dir
    relative_path = os.path.relpath(series_df.fname.iloc[0], data_dir)
    save_path = os.path.join(data_dir, destination_folder, os.path.dirname(relative_path))

    if not os.path.exists(save_path):
        os.makedirs(save_path)

    if write_labels:
        #print('writing new data into', save_path)
        write_labels_into_dicom(sorted_series, label_num=predicted_series_class,
                            conf_num=np.round(predicted_series_confidence, 3), path=save_path)

    return sorted_series
    



In [ ]:
def pipeline_new_image_df(data_dir, dest_name='modified', write_labels = True):
    # create the df of image data    
    _, df = get_dicoms(data_dir)
    df1 = df.copy()
    ## manipulate df prior to evaluation
    df1 = expand_filename(df1, ['blank', 'filename', 'series', 'exam', 'patientID'])
    df1.drop(columns='blank', inplace=True)
    df1['file_info']=df1.fname
    df1['img_num'] = df1.file_info.apply(extract_image_number)
    df1['contrast'] = df1.apply(detect_contrast, axis=1)
    df1['plane'] = df1.apply(compute_plane, axis=1)
    df1['series_num'] = df1.series.apply(lambda x: str(x).split('_')[-1])
    #print('columns before preprocess are', df1.columns)

    df1 = preprocess(df1)
   # print('after preprocessin exam is in columns?', ('exam' in df1.columns))
    #print('after preprocessing series is in columns?', ('series' in df1.columns))
    #process the batch of studies
    processed_frame = process_batch(df1, data_dir, dest_name, write_labels)

    return processed_frame


In [ ]:
def select_one_image(series, selection_fraction=0.5):
    sorted_series = series.sort_values()
    #print(sorted_series)
    index = int(len(sorted_series) * selection_fraction)
    print(index, len(sorted_series))
    return sorted_series.iloc[index]

In [ ]:
def mask_one_from_series2(df, selection_fraction=0.5):
    def select_one_image(series):
        sorted_series = series.sort_values()
        index = int(len(sorted_series) * selection_fraction)
        return sorted_series.iloc[index]

    df = df.sort_values(by=['patientID', 'series', 'file_info'])

    grouped_df = df.groupby(['patientID', 'series'])
    selected_rows = grouped_df.agg(select_one_image).reset_index()
    
    return selected_rows

# Example usage
# data = {'patientID': [1, 1, 1, 2, 2, 3, 3, 3, 3],
#         'series': [1, 1, 2, 1, 2, 1, 1, 2, 2],
#         'file_info': ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I']}

# df = pd.DataFrame(data)
# selected_rows = mask_one_from_series(df, selection_fraction=0.5)
# print(selected_rows)

In [ ]:


# def filter_out_mv(df):
#     # Create a boolean mask to identify rows with MultiValue objects in any column
#     mask = df.applymap(lambda x: isinstance(x, MultiValue)).any(axis=1)
#     #print(df[mask])
#     # Invert the mask to select rows without MultiValue objects
#     rows_without_multivalue = ~mask

#     # Filter out rows with MultiValue objects in any column
#     filtered_df = df[rows_without_multivalue]

#     return mask, filtered_df

#from pydicom.multival import MultiValue
# def compute_plane_new(row):
#     '''
#     Computes the plane of imaging from the direction cosines provided in the `ImageOrientationPatient` field.
#     The format of the values in this field is: `[x1, y1, z1, x2, y2, z2]`,
#     which correspond to the direction cosines for the first row and column of the image pixel data.
#     '''
#     planes = ['sag', 'cor', 'ax']
#     if 'ImageOrientationPatient1' in row.keys():
#         dircos = [v for k, v in row.items() if 'ImageOrientationPatient' in k]
#     else:
#         dircos = row['ImageOrientationPatient']

#         # Handle MultiValue objects by converting them to a list of floats
#         if isinstance(dircos, MultiValue):
#             dircos = [float(x) for x in dircos]

#     # Check if dircos has the expected length
#     if not isinstance(dircos, float) and len(dircos) == 6:
#         dircos = np.array(dircos).reshape(2, 3)
#         pnorm = abs(np.cross(dircos[0], dircos[1]))
#         return planes[np.argmax(pnorm)]
#     else:
#         return 'unknown'



In [ ]:

  # for idx, image_file in enumerate(selected_images):
            #     with cols[idx % 4]:
            #         dcm_data = pydicom.dcmread(image_file)
            #         image = Image.fromarray(dcm_data.pixel_array)
            #         image = image.convert("L")  # Convert the image to grayscale mode
            #         st.image(image, caption=os.path.basename(image_file), use_column_width=True)

# def apply_window_level(image, window_center, window_width):
#     min_value = window_center - window_width // 2
#     max_value = window_center + window_width // 2
#     image = np.clip(image, min_value, max_value)

#     def normalize_array(arr):
#         arr_min, arr_max = arr.min(), arr.max()
#         return (arr - arr_min) * 255 / (arr_max - arr_min)

#     return normalize_array(image)

# start_folder = '/volumes/cm7/start_folder'
# dicom_files = load_dicom_images(start_folder)

# patient_list = sorted(list(set([os.path.basename(os.path.dirname(f)) for f in dicom_files])))
# selected_patient = st.sidebar.selectbox("Select a Patient", patient_list)

# exam_list = sorted(list(set([os.path.basename(os.path.dirname(os.path.dirname(f))) for f in dicom_files if selected_patient in f])))
# selected_exam = st.sidebar.selectbox("Select an Exam", exam_list)

# series_list = sorted(list(set([os.path.basename(f) for f in dicom_files if selected_exam in f])))
# selected_series = st.selectbox("Select a Series", series_list)

# selected_images = [f for f in dicom_files if selected_series in f]
# selected_images.sort(key=lambda x: os.path.basename(x))  # Sort images within each series by filename

# # Process images button
# process_images = st.button("Process Images")

# if process_images:
#     # Classify the series with your machine learning model
#     #predicted_type = classify_series(selected_images)  # Replace with the appropriate function call
#     st.write(f"Predicted Type: not implemented yet")

# image_idx = st.slider("Select Image Index", 0, len(selected_images) - 1, 0)

# window_center = st.slider("Window Center", 0, 4096, 2048)
# window_width = st.slider("Window Width", 0, 4096, 4096)

# with st.container():
#     image_file = selected_images[image_idx]
#     try:
#         dcm_data = pydicom.dcmread(image_file)
#         image = dcm_data.pixel_array
#         image = apply_window_level(image, window_center, window_width)
#         image = Image.fromarray(np.uint8(image))
#         st.image(image, caption=os.path.basename(image_file), use_column_width=True)
#     except Exception as e:
#         pass  # Ignore the file and do nothing






# # # Function to load DICOM files into a dataframe
# # def load_dicom_data(folder):
# #     data = []
# #     for root, _, files in os.walk(folder):
# #         for file in files:
# #             if file.lower().endswith(".dcm"):
# #                 try:
# #                     dcm_file_path = os.path.join(root, file)
# #                     dcm_data = dcmread(dcm_file_path)
# #                     data.append(
# #                         {
# #                             "patient": dcm_data.PatientName,
# #                             "exam": dcm_data.StudyDescription,
# #                             "file_path": dcm_file_path,
# #                         }
# #                     )
# #                 except Exception as e:
# #                     st.warning(f"Error reading DICOM file {file}: {e}")

# #     return pd.DataFrame(data)

# # start_folder = "/volumes/cm7/start_folder"

# # if os.path.exists(start_folder) and os.path.isdir(start_folder):
# #     folder = st.sidebar.selectbox("Select a folder:", os.listdir(start_folder), index=0)
# #     selected_folder = os.path.join(start_folder, folder)

# #     if os.path.exists(selected_folder) and os.path.isdir(selected_folder):
# #         dicom_df = load_dicom_data(selected_folder)

# #         if not dicom_df.empty:
# #             st.subheader("Available Studies")
# #             unique_studies = dicom_df[["patient", "exam"]].drop_duplicates()
# #             study_list = [f"{row.patient} - {row.exam}" for _, row in unique_studies.iterrows()]

# #             selected_study = st.selectbox("Select a study:", study_list)

# #             selected_patient, selected_exam = selected_study.split(" - ")

# #             selected_images = dicom_df[
# #                 (dicom_df["patient"] == selected_patient) & (dicom_df["exam"] == selected_exam)
# #             ]["file_path"].tolist()

# #             st.subheader("Selected Study Images")
# #             cols = st.columns(4)

# #             for idx, image_file in enumerate(selected_images):
# #                 with cols[idx % 4]:
# #                     image = Image.open(image_file)
# #                     st.image(image, caption=os.path.basename(image_file), use_column_width=True)
# #         else:
# #             st.warning("No DICOM files found in the folder.")
# # else:
# #     st.error("Invalid start folder path.")



# # # def dir_selector(folder_path='/volumes/cm7/archived/modified/CmmDemoCase6/'):
# # #     while True:
# # #         folder_list = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]
# # #         folder_list.insert(0, '..')  # Add a ".." option to go back up one level
# # #         unique_key = 'dir_selector_' + folder_path.replace(os.path.sep, '_')
# # #         selected_folder = st.sidebar.selectbox('Select a folder:', folder_list, index=0, key=unique_key)

# # #         if selected_folder == '..':
# # #             folder_path = os.path.dirname(folder_path)
# # #         else:
# # #             folder_path = os.path.join(folder_path, selected_folder)
# # #             break

# # #     return folder_path
    
# # # def plot_slice(vol, slice_ix):
# # #     fig, ax = plt.subplots()
# # #     plt.axis('off')
# # #     selected_slice = vol[slice_ix, :, :]
# # #     ax.imshow(selected_slice, origin='lower', cmap='gray')
# # #     return fig
    

# # # st.sidebar.title('DieSitCom')
# # # dirname = dir_selector()

# # # if dirname is not None:
# # #     try:
# # #         reader = sitk.ImageSeriesReader()
# # #         dicom_names = reader.GetGDCMSeriesFileNames(dirname)
# # #         reader.SetFileNames(dicom_names)
# # #         reader.LoadPrivateTagsOn()
# # #         reader.MetaDataDictionaryArrayUpdateOn()
# # #         data = reader.Execute()
# # #         img = sitk.GetArrayViewFromImage(data)
    
# # #         n_slices = img.shape[0]
# # #         slice_ix = st.sidebar.slider('Slice', 0, n_slices, int(n_slices/2))
# # #         output = st.sidebar.radio('Output', ['Image', 'Metadata'], index=0)
# # #         if output == 'Image':
# # #             fig = plot_slice(img, slice_ix)
# # #             plot = st.pyplot(fig)
# # #         else:
# # #             metadata = dict()
# # #             for k in reader.GetMetaDataKeys(slice_ix):
# #                 metadata[k] = reader.GetMetaData(slice_ix, k)
# #             df = pd.DataFrame.from_dict(metadata, orient='index', columns=['Value'])
# #             st.dataframe(df)
# #     except RuntimeError:
# #         st.text('This does not look like a DICOM folder!')




In [ ]:
# # Backup of all of utils so i can clean up

# import pandas as pd
# import numpy as np
# import pydicom
# from datetime import datetime

# from pydicom.dataset import Dataset as DcmDataset
# from pydicom.tag import BaseTag as DcmTag
# from pydicom.multival import MultiValue as DcmMultiValue
# from pydicom.datadict import keyword_for_tag
# import matplotlib.pyplot as plt
# from pathlib import Path
# from torch.optim import lr_scheduler
# import torch.backends.cudnn as cudnn
# import torchvision
# from torchvision import datasets, models, transforms
# from torch.utils.data import Dataset, DataLoader
# from torchvision.transforms import ToTensor
# import os, sys, glob, re
# from joblib import dump, load
# from fastai.basics import delegates
# from fastcore.parallel import parallel
# from fastcore.utils import gt
# from fastcore.foundation import L
# import sklearn
# from sklearn.model_selection import train_test_split

# from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, recall_score, ConfusionMatrixDisplay
# from sklearn.linear_model import LogisticRegression

# ### local imports ###
# from config import file_dict, abd_label_dict, classes, column_lists, feats
# from config import val_list, train_val_split_percent, random_seed, data_transforms
# from config import sentence_encoder, series_description_column
# from sklearn.preprocessing import MultiLabelBinarizer
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# from sklearn.preprocessing import MinMaxScaler

# ### gets the dicom files from a provided directory ###
# def get_dicoms(path, first_dcm=False, **kwargs):
#     "Walk `path` to get DICOM file names from specific extensions, then read files into a `pandas.DataFrame`. If `first_dcm=True`, only read first file from each folder."
#     fns = L()
#     extension_list=['.dcm','.dicom','.dcim','.ima']
#     print("Finding DICOM files. This may take a few minutes.")
#     if first_dcm:
#         for r, d, f in os.walk(path):
#             if f:
#                 if Path(f[0]).suffix.lower() in extension_list:
#                     fns.append(Path(f'{r}/{f[0]}'))
#     else:
#         fns = L()
#         for r, d, fs in os.walk(path):
#             for f in fs:
#                 if Path(f).suffix.lower() in extension_list:
#                     fns.append(Path(f'{r}/{f}'))
#     print("Reading DICOM files with extensions .dcm, .dicom, .dcim, or .ima. This may take a few minutes, depending on the number of files to read...")
#     df = pd.DataFrame.from_dicoms(fns, **kwargs)
#     return fns, df

# ### Reads a DICOM file and returns the corresponding pydicom.Dataset ###
# def dcmread(fn: Path, no_pixels=True, force=True):
#     return pydicom.dcmread(str(fn), stop_before_pixels=no_pixels, force=force)

# def cast_dicom_special(x):
#     cls = type(x)
#     if not cls.__module__.startswith('pydicom'): return x
#     if cls.__base__ == object: return x
#     return cls.__base__(x)

# def split_elem(res, k, v):
#     if not isinstance(v, DcmMultiValue): return
#     for i, o in enumerate(v): res[f'{k}{"" if i == 0 else i}'] = o


# def as_dict(self: DcmDataset, filt=True, split_multi=False):
#     if filt:
#         vals = [self[o] for o in self.keys() if self[o].keyword in column_lists['dicom_cols']]
        
#     else:
#         vals = [self[o] for o in self.keys()]
#     items = [(v.keyword, v.value.name) if v.keyword == 'SOPClassUID' else (v.keyword, v.value) for v in vals]
#     res = dict(items)
#     res['fname'] = self.filename
#     if split_multi:
#         for k, v in items: split_elem(res, k, v)
#         for k in res: res[k] = cast_dicom_special(res[k])
#     return res
# DcmDataset.as_dict = as_dict

# # def _dcm2dict(fn, excl_private=False, **kwargs):
# #     ds = fn.dcmread(**kwargs)
# #     if excl_private: ds.remove_private_tags()
# #     return ds.as_dict(**kwargs)

# def dcm2dict(fn, excl_private=False, **kwargs):
#     ds = dcmread(fn, **kwargs)
#     if excl_private: ds.remove_private_tags()
#     return ds.as_dict(**kwargs)

# # def dcm2dict2(dcm_file, excl_private = False, **kwargs):
# #     dcm_data = dcmread(dcm_file, **kwargs)
# #     if excl_private: dcm_data.remove_private_tags()
# #     dcm_dict = {keyword_for_tag(tag): dcm_data.get(tag) for tag in dcm_data.keys()}
# #     return dcm_dict



# @delegates(parallel)
# def from_dicoms(cls, fns, n_workers=0, **kwargs):
#     return pd.DataFrame(parallel(dcm2dict, fns, n_workers=n_workers, **kwargs))
# pd.DataFrame.from_dicoms = classmethod(from_dicoms)

# # def _from_dicoms(cls, fns):
# #     dicts = [dcm2dict(fn) for fn in fns]  # Process the files sequentially
# #     return pd.DataFrame(dicts)
# # pd.DataFrame.from_dicoms = classmethod(_from_dicoms)

# def get_series_fp(fn): return Path(fn).parent

#  ### takes the contents of the dataframe column with path/filenames and converts pieces into separate df columns ###   
# def expand_filename_into_columns(df, cols):
#     for iterator in range(len(cols)):
#         df[cols[iterator]] = df['fname'].astype(str).apply(lambda x: x.split('/')[iterator])
#     return df

# ### another version which goes backwards from the end of the filename
# def expand_filename(df, cols):
#     for iterator in range(len(cols)):
#         df[cols[iterator]] = df['fname'].astype(str).apply(lambda x: x.split('/')[-iterator])
#     return df

# def extract_image_number(filename):
#     pattern = r'-([0-9]+)\.dcm$'
#     match = re.search(pattern, filename)
#     if match:
#         return int(match.group(1))
#     else:
#         return None


# def compute_plane(row):
#     '''
#     Computes the plane of imaging from the direction cosines provided in the `ImageOrientationPatient` field.
#     The format of the values in this field is: `[x1, y1, z1, x2, y2, z2]`,
#     which correspond to the direction cosines for the first row and column of the image pixel data.
#     '''
#     planes = ['sag', 'cor', 'ax']
#     if 'ImageOrientationPatient1' in row.keys():
#         dircos = [v for k, v in row.items() if 'ImageOrientationPatient' in k]
#     else:
#         dircos = row['ImageOrientationPatient']

#         # Handle MultiValue objects by converting them to a list of floats
#         if isinstance(dircos, DcmMultiValue):
#             dircos = [float(x) for x in dircos]

#     # Check if dircos has the expected length
#     if not isinstance(dircos, float) and len(dircos) == 6:
#         dircos = np.array(dircos).reshape(2, 3)
#         pnorm = abs(np.cross(dircos[0], dircos[1]))
#         return planes[np.argmax(pnorm)]
#     else:
#         return 'unknown'

# #_re_extra_info = re.compile(r'[<\([].*?[\]\)>]')

# def rm_extra_info(t):
#     "Remove extraneous info in closures"
#     return re.compile(r'[<\([].*?[\]\)>]').sub('', t).strip()


# def detect_contrast(row):
    
#     #if entry in contrastbolusagent contains a value
#     try:
#         c = row['ContrastBolusAgent']
#         if type(c) == str: return 1
#     except KeyError:
#         pass
#     # heuristic based on series description text
#     sd = rm_extra_info(str(row['SeriesDescription']).lower())
#     _c = re.compile(r'(\+-?c|post|with|dyn|portal|equilibrium|hepatobiliary|delayed)')
#     if _c.search(sd): return 1

#     return 0



# def _find_seq(sd):
#     if _t1.search(sd):
#         if _spgr.search(sd): return 'spgr'
#         if _t1_in.search(sd): return 'in phase'
#         if _t1_out.search(sd): return 'opposed phase'
#         if _water.search(sd): return 'dixon water'
#         if _fat.search(sd): return 'dixon fat'    
#         if _pv.search(sd): return 'portal venous'
#         if _eq.search(sd): return 'equilibrium'
#         if _art.search(sd): return 'early dynamic'
#         if _delayed.search(sd): return 'hepatobiliary'
#         else: return 't1'
#     if _t1_in.search(sd): return 'in phase'
#     if _t1_out.search(sd): return 'opposed phase'
#     if _water.search(sd): return 'dixon water'
#     if _fat.search(sd): return 'dixon fat'    
#     if _pv.search(sd): return 'portal venous'
#     if _eq.search(sd): return 'equilibrium'
#     if _art.search(sd): return 'early dynamic'
#     if _delayed.search(sd): return 'hepatobiliary'
# #    if _spgr.search(sd): return 'spgr'
#     if _t2.search(sd):
#         if _flair.search(sd): return 'flair'
#         elif _swi.search(sd): return 'swi'
#         else: return 't2'
#     if _flair.search(sd): return 'flair'
#     if _swi.search(sd): return 'swi'
#     if _dwi.search(sd): return 'dwi'
#     if _adc.search(sd): return 'dwi'
#     if _eadc.search(sd): return 'dwi'
#     if _mra.search(sd): return 'mra'
#     if _loc.search(sd): return 'loc'
#     if _other.search(sd): return 'other'
#     return 'unknown'



# def _make_col_binary(df, col):
#     s = df[col].isna()
#     if any(s):
#         df[col] = s.apply(lambda x: 0 if x else 1)
#     else:
#         targ = df.loc[0, col]
#         df[col] = df[col].apply(lambda x: 0 if x == targ else 1)

# def make_binary_cols(df, cols):
#     df1 = df.copy()
#     for col in cols:
#         if col in df.columns:
#             _make_col_binary(df1, col)
#         else: 
#             df1[col]=0
#             _make_col_binary(df1,col)
#     return df1

# def rescale_cols(df, cols):
#     df1 = df.copy()
#     scaler = MinMaxScaler()
#     df1[cols] = scaler.fit_transform(df1[cols])
#     return df1.fillna(0)

# def get_dummies(df, cols=column_lists['dummies'], prefix=column_lists['d_prefixes']):
#     df1 = df.copy()
#     for i, col in enumerate(cols):
#         df1[col] = df1[col].fillna('NONE')
#         mlb = MultiLabelBinarizer()
#         df1 = df1.join(
#             pd.DataFrame(mlb.fit_transform(df1.pop(col)), columns=mlb.classes_).add_prefix(f'{prefix[i]}_')
#         )
#     return df1

# #get labels from a text file in the format of comma deliminted four columns ()
# def labels_from_file(label_path, column_names):

#     label_df = pd.read_csv(label_path,header=None)
#     label_df.columns=column_names

#     return label_df

# def preprocess(df, keep= column_lists['keep'], dummies= column_lists['dummies'], d_prefixes= column_lists['d_prefixes'], binarize= column_lists['binarize'], rescale= column_lists['rescale']):
#    #Preprocess metadata for Random Forest classifier to predict sequence type
#     print("Preprocessing metadata for Random Forest classifier.")
#     df1 = exclude_other(df)
#     print(f"Have received {df1.shape[0]} entries.")
    
#     # Only keep columns that are both in the DataFrame and the 'keep' list
#     df1 = df1[[col for col in keep if col in df1.columns]]
    
#     if 'PixelSpacing' in df1.columns and df1['PixelSpacing'].any:
#         df1['PixelSpacing'] = df1['PixelSpacing'].apply(lambda x: x[0])
    
#     # Only get dummies for columns that are in the DataFrame
#     dummies = [col for col in dummies if col in df1.columns]
#     df1 = get_dummies(df1, dummies, d_prefixes)
    
#     # Only make binary columns for columns that are in the DataFrame
#     binarize = [col for col in binarize if col in df1.columns]
#     df1 = make_binary_cols(df1, binarize)
    
#     # Only rescale columns that are in the DataFrame
#     rescale = [col for col in rescale if col in df1.columns]
#     df1 = rescale_cols(df1, rescale)
    
#     for f in feats:
#         if f not in df1.columns:
#             df1[f] = 0
            
#     return df1



# # def preprocess2(df, keep=column_lists['keep'], dummies=column_lists['dummies'], d_prefixes=column_lists['d_prefixes'], binarize=column_lists['binarize'], rescale=column_lists['rescale']):
# #     "Preprocess metadata for Random Forest classifier to predict sequence type"
# #     print("Preprocessing metadata for Random Forest classifier.")
# #     df1 = exclude_other(df)
# #     print(f"Have received {df1.shape[0]} entries.")
    
# #     # Only keep columns that are both in the DataFrame and the 'keep' list
# #     df1 = df1[[col for col in keep if col in df1.columns]]
    
# #     if 'PixelSpacing' in df1.columns and df1['PixelSpacing'].any:
# #         df1['PixelSpacing'] = df1['PixelSpacing'].apply(lambda x: x[0])
    
# #     # Only get dummies for columns that are in the DataFrame
# #     dummies = [col for col in dummies if col in df1.columns]
# #     df1 = get_dummies(df1, dummies, d_prefixes)
    
# #     # Only make binary columns for columns that are in the DataFrame
# #     binarize = [col for col in binarize if col in df1.columns]
# #     df1 = make_binary_cols(df1, binarize)
    
# #     # Only rescale columns that are in the DataFrame
# #     rescale = [col for col in rescale if col in df1.columns]
# #     df1 = rescale_cols(df1, rescale)
    
# #     for f in feats:
# #         if f not in df1.columns:
# #             df1[f] = 0
            
# #     return df1


# def convert_labels_from_file(label_df):
#     labels=label_df.copy()
#     labels['GT label'] = labels['label_code'].astype(str).apply(lambda x: abd_label_dict[x]['short'])
#     labels['GT plane'] = labels['label_code'].astype(str).apply(lambda x: abd_label_dict[x]['plane'])
#     labels['GT contrast'] = labels['label_code'].astype(str).apply(lambda x: abd_label_dict[x]['contrast'])
#     labels['patientID'] = labels['patientID'].astype(str)
# #    labels['Parent_folder'] = labels['fname'].astype(str).apply(lambda x: x.split('/')[0])
# #    labels['patientID'] = labels['fname'].astype(str).apply(lambda x: x.split('/')[1]).astype(int)
# #    labels['exam'] = labels['fname'].astype(str).apply(lambda x: x.split('/')[2])
# #    labels['series'] = labels['fname'].astype(str).apply(lambda x: x.split('/')[3])
    
#     return labels

# def expand_filename_into_columns(df, cols):
#     for iterator in range(len(cols)):
#         df[cols[iterator]] = df['fname'].astype(str).apply(lambda x: x.split('/')[iterator])
#     return df

# def train_setup(df, preproc=True):
#     "Extract labels for training data and return 'unknown' as test set"
#     if preproc:
#         df1 = preprocess(df)
#         labels = extract_labels(df1)
#         df1 = df1.join(labels[['plane', 'contrast', 'seq_label']])
#     else:
#         df1 = df.copy()
#     filt = df1['seq_label'] == 'unknown'
#     train = df1[~filt].copy().reset_index(drop=True)
#     test = df1[filt].copy().reset_index(drop=True)
#     y, y_names = pd.factorize(train['seq_label'])
#     return train, test, y, y_names

# def train_setup_abdomen(df, cols=['patientID','exam','series'], preproc=False, need_labels=False):

#     if preproc:
#         df1=preprocess(df)
        
#     else:
#         df1=df.copy()
    
#     if need_labels:

#         labels = extract_labels(df1)
#         df1 = df1.merge(labels, on=cols)
 
#     length = df1.shape[0]

#     #gkf = GroupKFold(n_splits=5)
#     #for train_set, val_set in gkf.split(df1, groups=df1['patientID']):
#     #    train, val = df1.loc[train_set], df1.loc[val_set]
   
#     train_set, val_set = next(GroupShuffleSplit(test_size=.20, n_splits=1, random_state = 42).split(df1, groups=df1['patientID']))

#     train = df1.iloc[train_set]
#     val = df1.iloc[val_set]
#     y, y_names = train['label_code'],train['GT label']
 
#     return train, val, y, y_names

# # def _get_meta_preds(clf, df, features, y_names=_y_names):
# #     y_pred = clf.predict(df[features])
# #     y_prob = clf.predict_proba(df[features])
# #     preds = pd.Series(y_names.take(y_pred))
# #     probas = pd.Series([y_prob[i][pred] for i, pred in enumerate(y_pred)])
# #     return pd.DataFrame({'seq_pred': preds, 'pred_proba': probas})

# # def predict_from_df(df, features=_features, thresh=0.8, model_path=_model_path, clf=None, **kwargs):
# #     "Predict series from `df[features]` at confidence threshold `p >= thresh`"
# #     if 'plane' not in df.columns:
# #         df1 = preprocess(df)
# #         labels = extract_labels(df1)
# #         df1 = df1.join(labels[['plane', 'contrast', 'seq_label']])
# #     else:
# #         df1 = df.copy()
# #     if clf:
# #         model_path = None
# #     else:
# #         clf = load(model_path)    
# #     df1 = df1.join(_get_preds(clf, df1, features, **kwargs))
# #     filt = df1['pred_proba'] < thresh
# #     df1['seq_pred'][filt] = 'unknown'
# #     return df1

# # def predict_from_folder(path, **kwargs):
# #     "Read DICOMs into a `pandas.DataFrame` from `path` then predict series"
# #     _, df = get_dicoms(path)
# #     return predict_from_df(df, **kwargs)

# # class Finder():
# #     "A class for finding DICOM files of a specified sequence type from a specific ."
# #     def __init__(self, path):
# #         self.root = path
# #         self.fns, self.dicoms = get_dicoms(self.root)
# #         self.dicoms = preprocess(self.dicoms)
# #         self.labels = extract_labels(self.dicoms)
# #         self.dicoms = self.dicoms.join(self.labels[['plane', 'contrast']])
        
# #     def predict(self,  model_path=_model_path, features=_features, ynames=_y_names, **kwargs):
# #         try:
# #             self.clf = load(model_path)
# #         except FileNotFoundError as e:
# #             print("No model found. Try again by passing the `model_path` keyword argument.")
# #             raise
# #         self.features = features
# #         self.ynames = ynames
# #         preds = self.clf.predict(self.dicoms[features])
# #         self.preds = ynames.take(preds)
# #         self.probas = self.clf.predict_proba(self.dicoms[features])
        
# #     def find(self, plane='ax', seq='t1', contrast=True, thresh=0.8, **kwargs):
# #         try:
# #             self.probas
# #         except AttributeError:
# #             print("Prediction not yet performed. Please run `Finder.predict()` and try again.")
# #             raise
# #         preds = np.argwhere(self.probas > 0.8)
# #         ind = preds[:, 0]
# #         pred_names = _y_names.take(preds[:, 1])
# #         df = pd.DataFrame(pred_names, index=ind, columns=['seq_pred'])
# #         df = self.dicoms[_output_columns].join(df)
# #         return df.query(f'plane == "{plane}" and seq_pred == "{seq}" and contrast == {int(contrast)}')
    


# def exclude_other(df):
#     if 'BodyPartExamined' not in df.columns: return df
#     other = ['SPINE', 'CSPINE', 'PELVIS', 'PROSTATE']
#     filt = df.BodyPartExamined.isin(other)
#     df1 = df[~filt].copy().reset_index(drop=True)
#     filt1 = df1.SeriesDescription.str.contains(r'(cervical|thoracic|lumbar)', case=False, na=False)
#     df2 = df1[~filt1].reset_index(drop=True)
#     filt2 = df2.SOPClassUID == "MR Image Storage"
#     return df2[filt2].reset_index(drop=True)  

# def load_pickled_dataset(train_file, test_file):
#   with open(train_file, 'rb') as f:
#     train_df = pickle.load(f)
#   with open(test_file, 'rb') as g:
#     test_df = pickle.load(g)

#   return train_df, test_df

# def create_val_dataset_from_csv(train_file, test_file, val=True, val_lists=val_list):
#     train_df = pd.read_csv(train_file)
#     test_df = pd.read_csv(test_file)
    
#     if val:
#         if val_lists:
#             val_df = train_df[train_df.patientID.isin(val_lists)]
#             train_df = train_df[~train_df.index.isin(val_df.index)] 
#         else:
#             train_set, val_set = next(GroupShuffleSplit(test_size=.20, n_splits=1, random_state = 42).split(train_df, groups=train_df['patientID']))
#             train_df, val_df = train_set, val_set
#         return train_df, val_df, test_df

#     else: 
#         return train_df, test_df


# def load_datasets_from_csv(train_csv, val_csv, test_csv):
#     train_df = pd.read_csv(train_csv)
#     val_df = pd.read_csv(val_csv)
#     test_df = pd.read_csv(test_csv)

#     return train_df, val_df, test_df


# # this function will select the image in the middle of each series of images, so that only a single image from each series is selected for training
# # There is one image from each series for each patient
# def shorten_df(df, selection_fraction = 0.5):
#   df1 = df.copy()
#   grouped_df = df.groupby(['patientID', 'series'])
#   sorted_df = grouped_df['file_info'].apply(lambda x: x.sort_values())
  
#   selected_filename = grouped_df['file_info'].apply(lambda x: x.sort_values().iloc[int(len(x)*selection_fraction)])

  
#   selected_filename = selected_filename.reset_index()
  
#   # perform merge and deal with duplicate/unnecessary columns
#   df1 = df1.merge(selected_filename, on=['patientID', 'series'], how='left') 
#   df_short = df1.drop(['file_info_x', 'img_num'], axis=1)
#   df_short = df_short.rename(columns = {'file_info_y': 'file_info'})
#   df_short.drop_duplicates(inplace=True)
#   df_short.reset_index(drop=True, inplace=True)
#   return df_short

# # like shorten_df but does not adjust the dataframe, just returns the selected filenames
# def mask_one_from_series(df, selection_fraction=0.5):
#     df1 = df.copy()
#     grouped_df = df.groupby(['patientID', 'series'])
#     sorted_df = grouped_df['file_info'].apply(lambda x: x.sort_values())
  
#     selected_rows = grouped_df['file_info'].apply(lambda x: x.sort_values().iloc[int(len(x)*selection_fraction)])
   
#     return selected_rows

# def prepare_df(df):
#     df1 = df.copy()
#     filenames = df1.file_info.tolist()
#     getdicoms = pd.DataFrame.from_dicoms(filenames)
#     merged = getdicoms.merge(df1, left_on='fname', right_on='file_info')
#     merged.drop(columns=['file_info'], inplace=True)
    
#     merged['contrast'] = merged.apply(detect_contrast, axis=1)
#     merged['plane'] = merged.apply(compute_plane, axis=1)
    
    
#     return merged



# def pool_arterial_labels(df, label_col='label'):
#     df1 = df.copy()
#     df1[label_col] = df1[label_col].apply(lambda x: 2 if x in [2,3,4,5] else x)
#     return df1



        
# #visualization of a batch of images
# def imshow(img, title):
#     img = torchvision.utils.make_grid(img, normalize=True)
#     npimg = img.numpy()
#     fig = plt.figure(figsize = (5, 15))
#     plt.imshow(np.transpose(npimg,(1,2,0)))
#     plt.title(title)

#     plt.axis('off')
#     plt.show()

# # produce high quality confusion matrix image and save it
# # still need to add labels
# def plot_and_save_cm(ytrue, ypreds, fname):
#     cm = confusion_matrix(ytrue, ypreds)
#     plt.figure(figsize=(25, 25))
#     plt.tight_layout()
#     ConfusionMatrixDisplay(cm).plot(cmap='Blues')
#     plt.savefig(fname + datetime.now().strftime('%Y%m%d') + ".png", dpi=300, bbox_inches='tight')

#     plt.show()

#     return plt


# #explore train.fit and kgroupcross validation
# def train_setup_abdomen_cross(df, cols=['patientID','exam','series'], need_preproc=False, need_labels=False):

#     if need_preproc:
#         df1=preprocess(df)
        
#     else:
#         df1=df.copy()
    
#     if need_labels:

#         labels = extract_labels(df1)
#         df1 = df1.merge(labels, on=cols)
 
#     length = df1.shape[0]

#     gkf = GroupKFold(n_splits=5)
#     for train_set, val_set in gkf.split(df1, groups=df1['patientID']):
#         #print(train_set, len(train_set), train_set.dtype)
#         #print(df1.iloc[train_set])
#         train_df, val = df1.iloc[train_set], df1.iloc[val_set]
#         y, y_names = train_df['label_code'],train_df['GT label']
        
#         clf_gkf = train_fit(train_df, y, features=preproc._features, fname='cross_from_notebook.skl' )
#         scores = cross_validate(clf_gkf, train_df[preproc._features], y, scoring=['precision_macro', 'recall_macro'])
#         print(scores)
#     #return train, val, y, y_names

# # adds the preds and probs to select rows from the original data frame (patient and study info)
# def make_results_df(preds, probs, true, df):
#     return pd.DataFrame({'preds': preds, 'true': true, 'probs': [row.tolist() for row in probs], 'patientID': df['patientID'], 'series_description': df['SeriesDescription'], 'contrast': df['contrast'], 'plane': df['plane']  })


    
# def display_and_save_results(y_pred, y_true, classes=classes, fn='', saveflag = True):
   
#     class_text_labels = [abd_label_dict[str(x)]['short'] for x in classes]
   
#      # Generate a classification report based on the true labels and predicted labels
#     print(classification_report(y_true, y_pred))

#     # Generate a confusion matrix based on the true labels and predicted labels
#     cm = confusion_matrix(y_true = y_true, y_pred = y_pred, labels=classes)

#     # Create a ConfusionMatrixDisplay object with the correct labels
#     cm_display = ConfusionMatrixDisplay(cm, display_labels=class_text_labels).plot(xticks_rotation = 'vertical', cmap='Blues')
#     plt.figure(figsize=(25, 25))
#     plt.tight_layout()
#     #ConfusionMatrixDisplay(cm, display_labels=class_text_labels).plot(xticks_rotation = 'vertical', cmap='Blues')
#     if saveflag:
#         plt.savefig("../assets/FigCM_"+fn+datetime.today().strftime('%Y%m%d')+".tif",dpi=300, bbox_inches = 'tight')     

#     return cm      

# def create_datasets(train_datafile, val_datafile, test_datafile):
#     # reads in the dataframes from csv
#     train_full = pd.read_csv(train_datafile)
#     val_full = pd.read_csv(val_datafile)
#     test_full = pd.read_csv(test_datafile)

#     # selects the middle image from each series for further evaluation
#     train = shorten_df(train_full)
#     val = shorten_df(val_full)
#     test = shorten_df(test_full)

#     # changes to the dataframe including adding contrast and computed plane columns
#     train_df = prepare_df(train)
#     val_df = prepare_df(val)
#     test_df = prepare_df(test)

#     return train_df, val_df, test_df


# # #grid search for hyperparameters for the metadata model
# # def train_fit_parameter_trial(train, y, features, fname='model-run.skl'):
# #     "Train a Random Forest classifier on `train[features]` and `y`, then save to `fname` and return."
# #     clf = RandomForestClassifier(n_jobs=2, random_state=0)
# #     clf.fit(train[features], y)
# #     print('Parameters currently in use:\n')
# #     pprint(clf.get_params())
    
    
# #     # Number of trees in random forest
# #     n_estimators = [int(x) for x in np.linspace(start = 20, stop = 500, num = 20)]
# #     # Number of features to consider at every split
# #     max_features = ['auto', 'sqrt']
# #     # Maximum number of levels in tree
# #     max_depth = [int(x) for x in np.linspace(10, 660, num = 10)]
# #     max_depth.append(None)
# #     # Minimum number of samples required to split a node
# #     min_samples_split = [2, 5, 10, 20]
# #     # Minimum number of samples required at each leaf node
# #     min_samples_leaf = [2, 4, 8]
# #     # Method of selecting samples for training each tree
# #     bootstrap = [True, False]
# #     random_grid = {'n_estimators': n_estimators,
# #                'max_features': max_features,
# #                'max_depth': max_depth,
# #                'min_samples_split': min_samples_split,
# #                'min_samples_leaf': min_samples_leaf,
# #                'bootstrap': bootstrap}
    
# #     clf_random = RandomizedSearchCV(estimator = clf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=0, n_jobs = -1)
# #     clf_random.fit(train[features], y)
# #     opt_clf = clf_random.best_estimator_
# #     pprint(clf_random.best_params_)
# #     pickle.dump(opt_clf, open(fname, 'wb'))
# #     #dump(clf_random, fname)
# #     return opt_clf